<a href="https://colab.research.google.com/github/CAU-CAPSTONE-2-Friendly-Keyboard/Models/blob/main/%08BadtextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install lime
!pip install xformers

In [ ]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import numpy as np

## Setting

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')
tokenizer = AutoTokenizer.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')


In [ ]:
batch_size = 256

In [ ]:
classifier = pipeline("text-classification", model=model,tokenizer=tokenizer,batch_size=batch_size)

In [ ]:
def mk_predlist(inps):
  outputs = classifier(inps)
  result = []
  for output in outputs:
    if output['label'] == 'bad_sen':
      result.append([output['score'], 1-output['score']])
    else:
      result.append([1-output['score'], output['score']])
  return np.array(result)

In [ ]:
explainer = LimeTextExplainer(class_names=['bad','ok'])

In [ ]:
test_text = ""

In [ ]:
classifier(test_text)[0]

In [ ]:
if classifier(test_text)[0]['label'] == 'bad_sen':
  print('비속어 있음')
  exp = explainer.explain_instance(test_text, mk_predlist,top_labels=1)
  print(exp.as_list(label=0))
else:
  print('비속어 없음')


In [ ]:
exp.as_list(label=0)[0][0]

In [ ]:
# badwords = []
# for word, score in exp.as_list():
  # if abs(score)>=0.2:
  #   badwords.append(word)
# print(badwords)

In [ ]:
exp.available_labels()